 # Introdução 
 
* River é uma biblioteca Python para machine learning online . Ela tem como objetivo ser a biblioteca mais amigável para fazer machine learning em dados de streaming. River é o resultado de uma fusão entre creme e scikit-multiflow.[¹](https://github.com/online-ml/river).  
As funções que são de nosso interesse de estudo pode ser encontrado no case abaixo: 
   * [Online machine learning on river.](https://github.com/online-ml/river/blob/main/river/linear_model/base.py#L105)

Fica mais evidente que esta é a função learn_one correta pois a biblioteca creme tem a função fi_one que segue os mesmos passos que a função learn_one e tem a mesma rotina sobre gradiente igual a função learn_one da biblioteca river.    
Além disso, a biblioteca river pode ser descrita com uma junção da biblioteca scikit learn e da biblioteca creme o que reforça ainda mais esse entendimento.   
 * Estas funções atualizam o estado interno do modelo ao usar a biblioteca river, e nosso objetivo é entender os fundamentos por tras destas funções.   
     * ````_learn_mode(x)````

     
     * ````_fit(x,y,w,get_grad)````

# Função _learn_mode

* Estes são os passos resumidos que seguem a função:   
    * Armazenamento dos Pesos Originais: Os pesos atuais do modelo são armazenados na variável weights.


    * __Configuração Temporária dos Pesos__: Os pesos do modelo são configurados temporariamente usando uma estrutura VectorDict, que inclui um inicializador e uma máscara.A nova configuração é armazenada novamente em self._weights. Esta nova configuração usa VectorDict, que é uma estrutura que contém:
        * __weights__ : Os pesos originais armazenados na linha anterior.  


        * __self.initializer__: Um inicializador, que provavelmente é uma função ou objeto que define como os pesos devem ser configurados inicialmente.  


        * __mask__: A máscara fornecida como argumento da função.    

        
    * __Uso da Máscara__: A máscara (mask) é utilizada para selecionar quais partes dos pesos serão alteradas ou focadas durante o processo. Por exemplo, em aprendizado de máquina, uma máscara pode ser usada para aplicar mudanças apenas a certos elementos dos pesos, ignorando outros. Nesse caso, o VectorDict vai aplicar a máscara sobre os pesos existentes, possivelmente modificando ou excluindo certas partes com base na máscara.

    * __Execução Temporária__: O yield permite que a execução temporariamente utilize esses pesos configurados.
    O comando yield transforma essa função em um "generator", o que significa que ela pode pausar a execução e depois ser retomada. Aqui, yield permite que o código que chama essa função (provavelmente outra parte do código do modelo) continue a execução usando a nova configuração temporária de pesos.

    * __Restauração dos Pesos Originais__: Independentemente do que acontecer, os pesos são restaurados ao seu estado original no bloco finally.  
    O bloco finally garante que, independentemente do que acontecer durante a execução do yield, os pesos do modelo (self._weights) serão restaurados ao seu estado original (armazenado na variável weights no início da função). Isso é crucial para garantir que as modificações temporárias não afetem o modelo de forma permanente.


In [ ]:
def _learn_mode(self, mask=None):
    weights = self._weights
    try:
        # enable the initializer and set a mask
        self._weights = utils.VectorDict(weights, self.initializer, mask)
        yield
    finally:
        self._weights = weights

# Função _fit

*  Ela é responsável por ajustar os pesos do modelo com base em um único exemplo de dados (x, y), aplicando técnicas de otimização, atualização dos pesos e intercepto, além de considerar penalidades como L1.  
* Estes são os passos resumidos da função:
   * __Preparação do Otimizador__: Permite que o otimizador faça qualquer preparação necessária antes de uma predição.   
      * Esta linha permite que o otimizador faça alguma preparação antes de que uma predição seja feita. O método look_ahead do otimizador é chamado, recebendo os pesos atuais do modelo (self._weights). Esse passo pode envolver, por exemplo, a configuração de parâmetros internos do otimizador que serão usados no cálculo do gradiente ou atualização dos pesos.   

   * __Cálculo do Gradiente__: Calcula os gradientes da perda com relação aos pesos e à predição.  
      * Ela retorna dois valores:   
         * __gradient__: O gradiente da perda em relação aos pesos do modelo.  
         * __loss_gradient__: O gradiente da perda em relação ao intercepto do modelo (caso haja um intercepto).  
      Esses gradientes são necessários para ajustar os pesos e o intercepto do modelo.  

   * __Atualização do Intercepto__: Ajusta o intercepto do modelo com base no gradiente da perda.    
      * O método _get_intercept_update é chamado para calcular o quanto o intercepto deve ser ajustado, baseado no loss_gradient. O sinal negativo (-=) indica que o modelo está ajustando o intercepto na direção oposta ao gradiente, um princípio comum em métodos de otimização como o gradiente descendente.  

   * __Atualização dos Pesos__: Usa o otimizador para atualizar os pesos com base no gradiente calculado.   
      * O método step do otimizador recebe os pesos atuais (self._weights) e o gradiente (gradient). O otimizador aplica a atualização aos pesos, geralmente envolvendo a multiplicação do gradiente por uma taxa de aprendizado e a subtração do resultado dos pesos.  

   * __Aplicação da Penalidade L1__: Se o parâmetro l1 do modelo for diferente de zero, isso indica que a regularização L1 deve ser aplicada. A regularização L1 adiciona uma penalidade à soma dos valores absolutos dos pesos, incentivando a esparsidade (muitos pesos se tornando zero).  
      * __Penalidade L1__: A penalidade cumulativa L1 (max_cum_l1) é atualizada. A penalidade é aumentada em função da taxa de aprendizado atual (self.optimizer.learning_rate) e do valor l1. Isso é feito após a atualização dos pesos para considerar qualquer adaptação na taxa de aprendizado.  

      * __Aplicação da Penalidade aos Pesos__: O método _update_weights é chamado, e esse método provavelmente ajusta os pesos do modelo para aplicar a penalidade L1 acumulada. Isso pode envolver zerar certos pesos que estão abaixo de um determinado limiar, promovendo a esparsidade.


   * __Retorno do Modelo__: Retorna o modelo atualizado.
      *  Isso permite o encadeamento de métodos,


In [ ]:
def _fit(self, x, y, w, get_grad):
    # Some optimizers need to do something before a prediction is made
    self.optimizer.look_ahead(w=self._weights)

    # Calculate the gradient
    gradient, loss_gradient = get_grad(x, y, w)

    # Update the intercept
    self.intercept -= self._get_intercept_update(loss_gradient)

    # Update the weights
    self.optimizer.step(w=self._weights, g=gradient)

    # Apply L1 cumulative penalty if applicable
    if self.l1 != 0.0:
        # This should be called after the learning_rate update in case of adaptive learning rate
        self.max_cum_l1 = self.max_cum_l1 + self.l1 * self.optimizer.learning_rate

        self._update_weights(x)

    return self

### Como Funciona a Penalidade L1?

A penalidade L1 é introduzida na função de custo (ou função de perda) do modelo, que o modelo tenta minimizar durante o treinamento. Para um modelo de regressão linear, a função de custo padrão é o erro quadrático médio (MSE), mas com a regularização L1, a função de custo se torna:

$
\text{Função de Custo L1} = \text{MSE} + \lambda \sum_{j=1}^{n} |w_j|
$

Onde:
-  $\lambda $ (lambda) é um hiperparâmetro que controla a força da regularização. Um valor maior de $\lambda $ aumenta a penalidade, enquanto um valor menor reduz a penalidade.
- $ w_j $ são os coeficientes (pesos) do modelo.

### Efeitos da Penalidade L1

1. **Esparsidade**: A característica mais marcante da penalidade L1 é que ela pode forçar alguns coeficientes $ w_j $ a se tornarem exatamente zero. Isso significa que o modelo efetivamente ignora algumas características (features) dos dados, tornando o modelo mais simples e mais fácil de interpretar.

2. **Seleção de Características**: Devido à esparsidade, a penalidade L1 pode ser usada como uma forma de seleção de características, onde características irrelevantes ou menos importantes são eliminadas do modelo. Isso pode ser útil em cenários com muitos atributos, onde se deseja identificar automaticamente quais são mais relevantes.

3. **Overfitting Reduzido**: A regularização L1 ajuda a prevenir o overfitting ao penalizar modelos complexos com muitos coeficientes não nulos. Isso força o modelo a ser mais simples e, portanto, mais generalizável para novos dados.


# Função learn_one

* Assim a função atualiza o estado interno do modelo.  
A função learn_one realiza o ajuste no modelo para apenas um exemplo de dados. Em um cenário de aprendizado online, esse processo seria repetido continuamente à medida que novos exemplos de dados chegassem, permitindo que o modelo se adapte progressivamente.

* __Definição da função__: A função learn_one recebe os seguintes parâmetros:
    * __x__: As características (ou atributos) do exemplo de dados.
    * __y__: O rótulo ou valor alvo correspondente ao exemplo.
    * __w__: Um peso opcional associado ao exemplo atual, com valor padrão de 1.0. Isso permite que exemplos diferentes possam ter mais ou menos influência na atualização do modelo.

* __Contexto de Aprendizado__: A função learn_one utiliza um contexto de aprendizado com a instrução with, que envolve o método _learn_mode.  
    * __learn_mode__(x): O método _learn_mode que vimos anteriormente é chamado aqui, e ele gerencia temporariamente o estado dos pesos do modelo. Ele faz isso ajustando os pesos com base em um inicializador e uma possível máscara, permitindo que o modelo execute as operações necessárias dentro deste contexto e depois restaure os pesos ao estado original.    

* __Ajuste do Modelo__: Dentro do contexto de _learn_mode, a função _fit é chamada. Esta linha é a que realmente ajusta os parâmetros do modelo (pesos e intercepto) com base no exemplo de dados fornecido.
    * __x__: As características do exemplo.
    * __y__: O valor alvo correspondente ao exemplo.
    * __w__: O peso associado ao exemplo.
    * __get_grad=self._eval_gradient_one__: A função que calcula o gradiente é passada como argumento para _fit. Aqui, self._eval_gradient_one é a função específica usada para calcular o gradiente para o exemplo atual. Isso é fundamental porque o gradiente é o que orienta a atualização dos pesos para minimizar o erro do modelo.

In [ ]:
def learn_one(self, x, y, w=1.0):
    with self._learn_mode(x):
        self._fit(x, y, w, get_grad=self._eval_gradient_one)
